# Documentation
---
## Motivating Example: Rosenbrock Function

Consider the following problem

$$
\begin{aligned}
\min f(x_1,x_2) &= (1-x_1)^2 + 100 (x_2-x^2_1)^2\\
s.t.\\
x_1 &\geq -1.5 \,.
\end{aligned}
$$

In order to solve it with ALADIN, we have to reformulate the problem.
First, we substitute $x_1$ and $x_2$ which gives us

$$
\begin{aligned}
\min f(y_1, y_2, y_3) &= (1-y_1)^2 + 100 (y_2-y^2_3)^2 \\
&= f(y_1) + f(y_2,y_3)\\
s.t.\\
y_1 &= y_3,\\
 y_2 & \geq -1.5 \,. 
\end{aligned}
$$

Second, we split up the problem as follows

$$
\begin{aligned}
\min f(\tilde{y}_1, \tilde{y}_2) &= f(\tilde{y}_1) + f(\tilde{y}_2) \\
&= (1-\tilde{y}_1(1))^2 + 100 (\tilde{y}_2(1)-\tilde{y}_2(2)^2)^2 \\
s.t.\\
[1] \tilde{y}_1 + [0\ -1] \tilde{y}_2 &=0,\\
-1.5 - y_2(1) &\leq 0,\\
\end{aligned}
$$

with $\tilde{y}_1 = [y_1]$ and $\tilde{y}_2 = [y_2\ y_3]^\top$.

Using the MATLAB symbolic toolbox, we can now define the split-up problem (see `rosenbrock_example.m`) as:

```Matlab
y1  =   sym('y1',[1,1],'real');
y2  =   sym('y2',[2,1],'real');

f1  =   (1-y1(1))^2;
f2  =   100*(y2(1)-y2(2)^2)^2;

h2  =   -1.5-y2(1);

A1  =   [1];
A2  =   [0, -1];
b   =   0;

lb1 =   [-inf];
lb2 =   [-inf; -inf];

ub1 =   [inf];
ub2 =   [inf; inf];
```

---
## Solving the Problem using ALADIN

To solve the problem implemented above, we first have to convert the the symbolic variables to MATLAB functions:

```Matlab
emptyfun =   @(x) [];

f1f      =   matlabFunction(f1,'Vars',{y1});
f2f      =   matlabFunction(f2,'Vars',{y2});

h1f      =   emptyfun;
h2f      =   matlabFunction(h2,'Vars',{y2});
```

Next, we set the solver options. For more information on available options see [ALADIN solver options](#Options)

```Matlab
maxit =   15;
rho   =   10;
mu    =   100;
eps   =   1e-4;

opts  =   struct('rho0',rho,'rhoUpdate',1,'rhoMax',5e3,'mu0',...
                 mu,'muUpdate',1,'muMax',1e5,'eps',eps,...
                 'maxiter',maxit,'actMargin',-1e-6,'hessian',...
                 'full','solveQP','MA57','reg','true',...
                 'locSol','ipopt','innerIter',2400,'innerAlg',...
                 'full','plot',false,'Hess','standard');
```

Finally, we solve the problem with ALADIN:

```Matlab
ffifun           =   {f1f,f2f};
[ggifun{1:N}]    =   deal(emptyfun);
hhifun           =   {h1f,h2f};
AA               =   {A1,A2};
yy0              =   {[-2],[-2;1]};
lam0             =   10*(rand(1)-0.5);
llbx             =   {lb1,lb2};
uubx             =   {ub1,ub2};
Sig              =   {eye(1),eye(2)};

[xoptAL, loggAL] =   run_ALADIN(ffifun,ggifun,hhifun,AA,yy0,...
                                      lam0,llbx,uubx,Sig,opts);
```

---
## <a name="Options"></a>ALADIN Solver Options

TODO